# Databáze

V této lekci si ukážeme, jak se dá z pandas připojit k databázi. To nám umožní psát v rámci Pythonu SQL dotazy a jako výsledky dostávat dataframes. Ukážeme si pár příkladů a porovnáme rozdíly mezi zpracováním dat v databázi a v Pandas.

Nejdřív si ale ukážeme nějakou další práci s textovými řetězci, která se nám bude hodit při psaní SQL dotazů.

## 1. Víceřádkové stringy a další formátování

### Víceřádkové stringy

SQL dotazy se v Pythonu píšou jako obyčejné stringy. Jupyter to neumí, ale chytřejší programy poznají že je ve stringu SQL a zvýrazní nám jeho syntaxi.

Pro snazší čitelnost je, stejně jako v klasickém SQL, zvykem psát dotazy rozdělené do více řádků. Začneme tedy ukázkou, jak psát v Pythonu víceřádkové stringy.

Pokud ukončíme řádek uprostřed obyčejného stringu, chápe to Python jako oddělení dvou příkazů, a začne si stěžovat na neukončené uvozovky.

In [17]:
tohle_nepujde = "SELECT sloupec
FROM tabulka"

print(tohle_nepujde)

SyntaxError: EOL while scanning string literal (<ipython-input-17-18a9d138385f>, line 1)

Můžeme tam sice přidat znak `\n` jako nový řádek, ale ten funguje jen po vypsání na obrazovku nebo do souboru. Čitelnosti kódu to nijak nepomůže.

In [16]:
tohle_neni_citelne = "SELECT sloupec\nFROM tabulka"

print(tohle_neni_citelne)

SELECT sloupec
FROM tabulka


Python umožňuje začít (a ukončit) string třemi uvozovkami. Takový string pak chápe jako víceřádkový.

In [18]:
tohle_je_ok = """
SELECT sloupec
FROM tabulka
"""

print(tohle_je_ok)


SELECT sloupec
FROM tabulka



Takový string dokonce zachovává odsazení.

In [20]:
odsazeny_string = """
SELECT sloupec1,
       sloupec2,
FROM tabulka
WHERE sloupec3 IN (
    SELECT sloupec4
    FROM tabulka2
)
"""

print(odsazeny_string)


SELECT sloupec1,
       sloupec2,
FROM tabulka
WHERE sloupec3 IN (
    SELECT sloupec4
    FROM tabulka2
)



Občas si lidé myslí, že tři uvozovky znamenají víceřádkový komentář. To ale není pravda, Python žádné víceřádkové komentáře nemá.

In [21]:
"""
Zde není komentář.

Zde už vůbec ne.
"""

'\nZde není komentář.\n\nZde už vůbec ne.\n'

Jak vidíme z výstupu Jupyteru, Python vytvořil víceřádkový string. Jen se neuložil do žádné proměnné a tak hned zanikl. Narozdíl od komentářů (které se při spuštění programu přeskakují) ale Python každý víceřádkový string vytvoří, což stojí nějakou práci navíc.

Teoreticky je taky možné v rámci víceřádkového (f-)stringu spustit libovolný kód, což by v komentáři určitě jít nemělo.

In [23]:
f"""
Zde není komentář.

Zkusíme dělit nulou: {1 / 0}.
"""

ZeroDivisionError: division by zero

### Další způsob formátování stringů

Pro formátování stringů jsme si už dříve ukázali f-stringy. Ty nám umožňují do stringu dodat hodnotu libovolného Pythonovského výrazu. Ačkoliv Python umí víc způsobů, f-stringy se používají nejčastěji, protože jsou nejčitelnější a nejrychlejší.

Jejich nevýhodou ale je, že nemůžeme stejný string využít víckrát, a pokaždé do něj vložit hodnoty jiných výrazů. Právě to se nám někdy hodí při psaní modulárních SQL dotazů.

Dejme tomu, že máme tabulku studentů.

<table>
    <tr><th>Jméno</th><th>Bydliště</th><th>Studijní obor</th></tr>
    <tr><td>František Novák</td><td>Praha</td><td>Historie</td></tr>
    <tr><td>Josef Krátký</td><td>Litomyšl</td><td>Matematika</td></tr>
    <tr><td>Petr Švec</td><td>Olomouc</td><td>Alchymie</td></tr>
    <tr><td>David Kovář</td><td>Znojmo</td><td>Hudba</td></tr>
    <tr><td>Prokop Novotný</td><td>Praha</td><td>Tanec</td></tr>
    <tr><td>Marek Černý</td><td>Plzeň</td><td>Medicína</td></tr>
</table>

Podobně máme tabulku profesorů.

<table>
    <tr><th>Jméno</th><th>Bydliště</th><th>Vydaných článků</th></tr>
    <tr><td>Jiří Volf</td><td>Brno</td><td>1</td></tr>
    <tr><td>Jan Kulhavý</td><td>Praha</td><td>40</td></tr>
    <tr><td>Michal Horák</td><td>Ostrava</td><td>3</td></tr>
    <tr><td>Ondřej Veselý</td><td>Plzeň</td><td>18</td></tr>
</table>

Chceme vybrat studenty i profesory, kteří žijí v Praze. V obou případech bude dotaz podobný.

In [25]:
studenti_praha = """
SELECT jmeno
FROM studenti
WHERE bydliste = 'Praha'
"""

profesori_praha = """
SELECT jmeno
FROM profesori
WHERE bydliste = 'Praha'
"""

Protože se oba dotazy liší jen názvem tabulky, bylo by mnohem hezčí napsat si obecný dotaz pro výběr jmen s bydlištěm v Praze, a podle potřeby do něj doplnit název tabulky.

In [26]:
bydliste_praha = """
SELECT jmeno
FROM {tabulka}
WHERE bydliste = 'Praha'
"""

In [29]:
print(bydliste_praha.format(tabulka="studenti"))


SELECT jmeno
FROM studenti
WHERE bydliste = 'Praha'



In [30]:
print(bydliste_praha.format(tabulka="profesori"))


SELECT jmeno
FROM profesori
WHERE bydliste = 'Praha'



Můžeme jít ještě dál, a i místo bydliště nechat jako parametr. Jen pozor na jednoduché uvozovky okolo hodnoty tohoto parametru.

In [31]:
jmeno_podle_bydliste = """
SELECT jmeno
FROM {tabulka}
WHERE bydliste = '{bydliste}'
"""

In [33]:
print(jmeno_podle_bydliste.format(tabulka="studenti", bydliste="Brno"))


SELECT jmeno
FROM studenti
WHERE bydliste = 'Brno'



## 2. SELECT

In [1]:
import pandas as pd
import snowflake.connector

Nejprve potřebujeme zadat jméno a heslo pro přístup do databáze. Formálně je potřeba i role, ale zde víme jak vypadá, tak ji dostaneme automaticky.

In [ ]:
name = ...  # Dopln svuj login jako string.
password = ...  # Dopln svoje heslo jako string.
role = "ROLE_" + name.upper()

Doplníme údaje o databázovém serveru a pomocí Snowflake connectoru navážeme spojení.

In [3]:
DATABASE = "COURSES"
HOST = "https://ip68917.eu-west-1.snowflakecomputing.com/"
SCHEMA = "SCH_CZECHITA"
WAREHOUSE = "COMPUTE_WH"
ACCOUNT = "ip68917.eu-west-1"

conn = snowflake.connector.connect(
    user=name,
    password=password,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA,
    role=role,
)

Vyzkoušíme jednoduchý SQL dotaz -- vybereme všechny státy z tabulky `country` ve schématu `SCH_CZECHITA`. Výsledkem je obyčejný Pandas dataframe.

In [9]:
sql = """
    SELECT *
    FROM SCH_CZECHITA.country
"""

pd.read_sql_query(sql, conn)

,ID,NAME
0,4,Afghanistan
1,5,Albania
2,6,Algeria
3,8,Angola
4,11,Argentina
...,...,...
132,603,United Kingdom
133,1001,Serbia
134,1002,Montenegro
135,1003,Kosovo


Protože jsme schéma uvedli už při připojení, nemusíme ho opakovat u každé tabulky.

In [7]:
sql = """
    SELECT *
    FROM country
"""

pd.read_sql_query(sql, conn)

## 3. JOIN

## 4. GROUP BY